In [1]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

sys.path.append("/global/homes/r/rtojeiro/SV/python")
import VI_utils as VI

import matplotlib.pyplot as plt 

pd.set_option('display.max_rows', 5)

In [2]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/'
tiles = ['67230']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']
#obs_db = utils_specviewer.make_targetdict(tiledir, petals=petals, tiles=tiles) # tiles = optional 
#del obs_db['67230-20200314']
#obs_db.keys()

In [3]:
VI_dir = os.environ['HOME']+'/SV/VI_files/SV0/ELG/'

save_filename = 'truth_table_ELG_v1.1_latest.csv'

### Read individual VI files and concatenate all "VI Issue" fields by TargetID

In [4]:
vi = VI.read_individual_VI(VI_dir)

Reading all individual VI files in /global/homes/r/rtojeiro/SV/VI_files/SV0/ELG/
desi-vi_SV0_ELG_tile67230_night20200315_1_ace.csv
desi-vi_SV0_ELG_tile67230_night20200315_7_CBA.csv
desi-vi_SV0_ELG_tile67230_night20200315_13_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_1_DRP.csv
desi-vi_SV0_ELG_tile67230_night20200315_17_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_18_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_23_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_1_CBA_deleteme.csv
desi-vi_SV0_ELG_tile67230_night20200315_50_SAE.csv
desi-vi_SV0_ELG_tile67230_night20200315_56_PNT.csv
desi-vi_SV0_ELG_tile67230_night20200315_24_CGS.csv
desi-vi_SV0_ELG_tile67230_night20200315_32_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_49_SAE.csv
desi-vi_SV0_ELG_tile67230_night20200315_1_SAE.csv
desi-vi_SV0_ELG_tile67230_night20200315_58_KAD.csv
desi-vi_SV0_ELG_tile67230_night20200315_32_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_29_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_15_TWL.c

In [5]:
vi

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment
0,35185808403072900,-1,0.33.0.dev3890,0,GALAXY,0.8266,PNT,4.0,--,0.8266,GALAXY,--
1,35185808403072904,-1,0.33.0.dev3890,0,GALAXY,1.5389,PNT,3.0,--,1.5389,GALAXY,Noisy spectra OII is secured
...,...,...,...,...,...,...,...,...,...,...,...,...
9645,35185766149655639,-1,0.33.0.dev3890,0,GALAXY,1.2557,CGS,4.0,--,--,--,--
9646,35185766149657122,-1,0.33.0.dev3890,0,GALAXY,1.6106,CGS,2.0,S,--,--,--


In [6]:
for i in range(len(vi)):
    if type(vi.loc[i]['VI issue']) is float:
        vi.loc[i,'VI issue'] = '--'
        print('found it! i = ', i)

found it! i =  8198
found it! i =  8199
found it! i =  8200
found it! i =  8201
found it! i =  8202
found it! i =  8203
found it! i =  8204
found it! i =  8205
found it! i =  8206
found it! i =  8207
found it! i =  8208
found it! i =  8209
found it! i =  8210
found it! i =  8211
found it! i =  8212
found it! i =  8213
found it! i =  8214
found it! i =  8215
found it! i =  8216
found it! i =  8217
found it! i =  8218
found it! i =  8219
found it! i =  8220
found it! i =  8221
found it! i =  8222
found it! i =  8223
found it! i =  8224
found it! i =  8225
found it! i =  8226
found it! i =  8227
found it! i =  8228
found it! i =  8229
found it! i =  8230
found it! i =  8231
found it! i =  8232
found it! i =  8233
found it! i =  8234
found it! i =  8235
found it! i =  8236
found it! i =  8237
found it! i =  8238
found it! i =  8239
found it! i =  8240
found it! i =  8241
found it! i =  8242
found it! i =  8243
found it! i =  8244
found it! i =  8245
found it! i =  8246
found it! i =  8247


In [7]:
#create new column - concatenates all entries in the "VI issue" field
vi['all VI issue'] = vi.groupby('TargetID')['VI issue'].transform(lambda x: '|'.join(x))


In [8]:
vi

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,all VI issue
0,35185808403072900,-1,0.33.0.dev3890,0,GALAXY,0.8266,PNT,4.0,--,0.8266,GALAXY,--,--|--|--|--
1,35185808403072904,-1,0.33.0.dev3890,0,GALAXY,1.5389,PNT,3.0,--,1.5389,GALAXY,Noisy spectra OII is secured,--|--|--|--
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9645,35185766149655639,-1,0.33.0.dev3890,0,GALAXY,1.2557,CGS,4.0,--,--,--,--,--|--|--|--
9646,35185766149657122,-1,0.33.0.dev3890,0,GALAXY,1.6106,CGS,2.0,S,--,--,--,--|--|--|S


In [9]:
len(vi['TargetID'].to_list())

9647

### Make truth table

Read all the partial truth table files and concatenate into one

In [10]:
filenames = ['truth_table_ELGs_sets11_15.txt' , 'truth_table_ELGs_sets5_10.txt', 'truth_table_ELGs_sets16_20.txt',
            'truth_table_ELGs_sets2_4_new.txt', 'truth_table_ELGs_sets21_25.txt', 'truth_table_ELGs_sets26_30.txt', 
            'truth_table_ELGs_sets31_35.txt', 'truth_table_ELGs_sets36_40.txt', 'truth_table_ELGs_sets41_45.txt', 
            'truth_table_ELGs_sets46_50.txt']

truth_table = pd.read_csv(VI_dir+filenames[0])
for i in range(1,len(filenames)):
    partial_table = pd.read_csv(VI_dir+filenames[i])
    truth_table = truth_table.append(partial_table, ignore_index=True,sort=False)


In [11]:
truth_table

,TargetID,Redrock z,Redrock spectype,best redshift,best spectype,vi_combined_flag,N_VI,all VI comments,merger comment,DELTACHI2,FIBER,FLUX_G,FLUX_R,FLUX_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z
0,35185754028116247,0.5481,GALAXY,0.5481,GALAXY,0.333333,3,no real features|--| fit not convincing (doub...,none,6.121645,846,0.592657,0.784296,1.518387,0.190289,0.251820,0.487521
1,35185754028116361,0.8179,GALAXY,0.8179,GALAXY,4.000000,3,--|--|--,none,528.739231,753,0.488526,1.075064,2.871854,0.329701,0.725549,1.938184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2448,35185796340255156,0.5027,GALAXY,0.5027,GALAXY,0.500000,4,Calib issue at 5800AA| No signal|--|Not phy...,none,49.307792,3032,0.517165,0.584758,1.613210,0.109430,0.123733,0.341350
2449,35185796340255256,0.9999,GALAXY,0.9999,GALAXY,3.500000,4,Calib issue at 5800AA|--|--|Broad [OII],none,132.567042,3394,0.347042,0.447607,1.855011,0.248966,0.321111,1.330773


### Add the 'all VI issue' field to the truth table

In [12]:
vi[['TargetID', 'all VI issue']]
vi_gp = vi[['TargetID', 'all VI issue']].groupby('TargetID')

In [13]:
truth_table = truth_table.merge(vi_gp.first(), on='TargetID', how='left')

In [14]:
truth_table

,TargetID,Redrock z,Redrock spectype,best redshift,best spectype,vi_combined_flag,N_VI,all VI comments,merger comment,DELTACHI2,FIBER,FLUX_G,FLUX_R,FLUX_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,all VI issue
0,35185754028116247,0.5481,GALAXY,0.5481,GALAXY,0.333333,3,no real features|--| fit not convincing (doub...,none,6.121645,846,0.592657,0.784296,1.518387,0.190289,0.251820,0.487521,R|--|R
1,35185754028116361,0.8179,GALAXY,0.8179,GALAXY,4.000000,3,--|--|--,none,528.739231,753,0.488526,1.075064,2.871854,0.329701,0.725549,1.938184,--|--|--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2448,35185796340255156,0.5027,GALAXY,0.5027,GALAXY,0.500000,4,Calib issue at 5800AA| No signal|--|Not phy...,none,49.307792,3032,0.517165,0.584758,1.613210,0.109430,0.123733,0.341350,--|--|--|--
2449,35185796340255256,0.9999,GALAXY,0.9999,GALAXY,3.500000,4,Calib issue at 5800AA|--|--|Broad [OII],none,132.567042,3394,0.347042,0.447607,1.855011,0.248966,0.321111,1.330773,--|--|--|--


### Add more information from the FIBERMAP and ZBEST files

In [15]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

tspec_df = tspec['TARGETID','ZWARN', 'ZERR' ].to_pandas()
tf_df = tf['TARGETID','MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z'].to_pandas()

tf_df = tf_df.rename(columns={"TARGETID": "TargetID"})
tspec_df = tspec_df.rename(columns={"TARGETID": "TargetID"})

truth_table = truth_table.merge(tf_df, how='left', on='TargetID')
truth_table = truth_table.merge(tspec_df, how='left', on='TargetID')

In [16]:
truth_table.keys()

Index(['TargetID', 'Redrock z', 'Redrock spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments',
       'merger comment', 'DELTACHI2', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z',
       'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'all VI issue',
       'MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z', 'ZWARN',
       'ZERR'],
      dtype='object')

### Rename columns

* TargetID -> TARGETID
* best redshift -> best z
* vi_combined_flag -> best flag


In [17]:
truth_table = truth_table.rename(columns={"TargetID":"TARGETID", "best redshift":"best z", "vi_combined_flag":"best flag"})

In [18]:
truth_table.keys()

Index(['TARGETID', 'Redrock z', 'Redrock spectype', 'best z', 'best spectype',
       'best flag', 'N_VI', 'all VI comments', 'merger comment', 'DELTACHI2',
       'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R',
       'FIBERFLUX_Z', 'all VI issue', 'MW_TRANSMISSION_G', 'MW_TRANSMISSION_R',
       'MW_TRANSMISSION_Z', 'ZWARN', 'ZERR'],
      dtype='object')

### re-ordering columns

In [19]:
truth_table = truth_table[['TARGETID', 'Redrock z', 'Redrock spectype', 'best z', 'best spectype',
       'best flag', 'N_VI', 'all VI issue', 'all VI comments', 'merger comment', 'DELTACHI2', 'ZWARN', 'ZERR','FIBER', 
        'FLUX_G', 'FLUX_R', 'FLUX_Z',
       'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'MW_TRANSMISSION_G',
       'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z']]

In [20]:
truth_table.keys()

Index(['TARGETID', 'Redrock z', 'Redrock spectype', 'best z', 'best spectype',
       'best flag', 'N_VI', 'all VI issue', 'all VI comments',
       'merger comment', 'DELTACHI2', 'ZWARN', 'ZERR', 'FIBER', 'FLUX_G',
       'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z',
       'MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z'],
      dtype='object')

In [21]:
print(VI_dir+save_filename)

/global/homes/r/rtojeiro/SV/VI_files/SV0/ELG/truth_table_ELG_v1.1_latest.csv


In [22]:
truth_table.to_csv(VI_dir+save_filename)

In [23]:
len(truth_table)

2450